# Project

### Importando as bibliotecas

In [1]:
import sys
sys.path.append('..')
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import params.consts as consts

# É possível fazer o projeto com o pyodbc também mas o pandas recomenda que seja usado o sqlalchemy

### Conectando com o banco de dados

In [2]:
driver = 'SQL+Server' # Passando o nome do programa a ser utilizado (com + onde é espaço)
server = 'GalaxyBook2-360' # Passando o nome do servidor/usuário. Para descobrir, basta digitar no cmd hostname
database = 'AdventureWorksDW2022' # Passando o nome do banco de dados a ser consultado

engine = create_engine(f'mssql+pyodbc://{server}/{database}?driver={driver}') # Fazendo a conexão com o banco de dados

print('Conexão realizada com sucesso.') # Confirmando que a conexão foi realizada com sucesso

Conexão realizada com sucesso.


### Consultando o banco de dados

In [3]:
sql_code = ''' 
    WITH cte AS (
        SELECT 
            AVG(fis.SalesAmount - fis.TotalProductCost) AS avg_profit,
            MIN(fis.SalesAmount - fis.TotalProductCost) AS min_profit,
            MAX(fis.SalesAmount - fis.TotalProductCost) AS max_profit
        FROM 
            FactInternetSales fis
        WHERE 
            YEAR(fis.OrderDate) = 2013
    )

    SELECT TOP (100000)
        fis.SalesOrderNumber AS 'order_number',
        fis.OrderDate AS 'order_date',
        dpc.EnglishProductCategoryName AS 'category',
        fis.CustomerKey AS 'customer_id',
        dc.FirstName + ' ' + dc.LastName AS 'name',
        REPLACE(REPLACE(dc.Gender, 'M','Male'), 'F', 'Female') AS 'gender',
        dg.EnglishCountryRegionName AS 'country',
        fis.OrderQuantity AS 'order_quantity',
        fis.SalesAmount AS 'sales_amount',
        fis.TotalProductCost AS 'total_product_cost',
        fis.SalesAmount - fis.TotalProductCost AS 'sales_profit'
    FROM
        cte,
        FactInternetSales fis
    INNER JOIN DimProduct dp ON fis.ProductKey = dp.ProductKey
        INNER JOIN DimProductSubcategory dps ON dp.ProductSubcategoryKey = dps.ProductSubcategoryKey
            INNER JOIN DimProductCategory dpc ON dps.ProductCategoryKey = dpc.ProductCategoryKey
    INNER JOIN DimCustomer dc ON fis.CustomerKey = dc.CustomerKey
        INNER JOIN DimGeography dg ON dc.GeographyKey = dg.GeographyKey
    WHERE 
        YEAR(fis.OrderDate) = 2013 AND 
        (fis.SalesAmount - fis.TotalProductCost) > cte.avg_profit AND
        dg.EnglishCountryRegionName = 'United States';
''' # Escrevendo o código em SQL para consultar os dados

df = pd.read_sql(sql_code, engine) # Consultando o banco de dados e criando um dataset com os dados extraídos

### Visualizando o dataset

In [4]:
df.head() # Exibindo o início do dataset

,order_number,order_date,category,customer_id,name,gender,country,order_quantity,sales_amount,total_product_cost,sales_profit
0,SO51520,2013-01-18,Bikes,11015,Chloe Young,Female,United States,1,2319.99,1265.6195,1054.3705
1,SO52512,2013-02-09,Bikes,11016,Wyatt Hill,Male,United States,1,2294.99,1251.9813,1043.0087
2,SO51610,2013-01-23,Bikes,11021,Destiny Wilson,Female,United States,1,2294.99,1251.9813,1043.0087
3,SO51556,2013-01-20,Bikes,11022,Ethan Zhang,Male,United States,1,2319.99,1265.6195,1054.3705
4,SO51593,2013-01-22,Bikes,11036,Jennifer Russell,Female,United States,1,2319.99,1265.6195,1054.3705


In [5]:
df.tail() # Exibindo o fim do dataset

,order_number,order_date,category,customer_id,name,gender,country,order_quantity,sales_amount,total_product_cost,sales_profit
3260,SO67690,2013-10-03,Bikes,29274,Miguel Green,Male,United States,1,742.35,461.4448,280.9052
3261,SO67466,2013-09-30,Bikes,29275,Taylor Thomas,Female,United States,1,742.35,461.4448,280.9052
3262,SO68872,2013-10-21,Bikes,29276,Sydney Flores,Female,United States,1,742.35,461.4448,280.9052
3263,SO68725,2013-10-19,Bikes,29278,Janet Scott,Female,United States,1,742.35,461.4448,280.9052
3264,SO68389,2013-10-14,Bikes,29279,Hailey Turner,Female,United States,1,742.35,461.4448,280.9052


### Salvando o dataset

In [6]:
df.to_csv(consts.DATASET, index=False) # Salvando o dataset em formato csv